<a href="https://colab.research.google.com/github/BrendanL72/Hospitalization-Risk/blob/main/arimamodek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install pmdarima

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dfadl = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/MobileEdgeTaskCodeVisitSummaryJanuary-AugustSortedAfterDeIdentify.xlsx')
dfadl.nunique()

In [ ]:
dfcrf = pd.read_excel('/content/gdrive/My Drive/Colab Notebooks/ClientReviewFormRemovingData.xlsx')
dfcrf.nunique()

In [ ]:
# clientarr = dfcrf['Masked Client ID'].unique()
# devicearr = df['device_id'].unique()
# #assign empty client column to sensor dataframe
# df['client'] = ''
# #fill df['client'] with random client ids corresponding to device id
# """for i in range(0, len(devicearr)):
#   for j in range(0, len(df)):
#     if(df['device_id'][j] == devicearr[i]):
#       df['client'][j] = dfcrf['Masked Client ID'][i]"""
# it = 0
# step = int(len(df)/dfcrf['Masked Client ID'].nunique())
# for i in range(0, len(df), step):
#   for j in range(0,step):
#     df['client'][i+j] = clientarr[it]
    
#   it = it + 1

In [ ]:
dfcrf.head()


,Time,Masked Client ID,Date of Review,COVID -19,Which Review is this,Insurance,Any Hospitalizations in the last 30 days,Note on Hospitalization (date and reason),How can we help prevent a return to hospital?,Have you fallen in:,...,Does your Caregiver carry out the care plan satisfactory,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes
0,2022-01-03 14:59:17,Client088,2022-01-03,By Phone,5 Day,Banner,No,NaN,NaN,No falls,...,Yes,NaN,Yes,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"ALL IS WELL FCG, SEEKING SUPPORT FOR MED APPTS..."
1,2022-01-04 13:12:24,Client172,2022-01-04,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,NaN,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,"CHECKING INTO DD SERVICES, HE HAS APPLIED."
2,2022-01-04 13:31:32,Client012,2022-01-04,By Phone,30 Day,UHC,Yes,COTTONWOOD HOSP DISCHARGED 12.31.2021,UNKNOWN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,ALL IS WELL
3,2022-01-06 11:56:52,Client022,2022-01-06,By Phone,90 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,NOT AT THIS TIME,NEW FCG IS APPLYING CARMEN HERNANDEZ
4,2022-01-10 11:03:08,Client087,2022-01-10,By Phone,30 Day,UHC,No,NaN,NaN,No falls,...,Yes,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,RAMS AND RAISE WALKER


In [ ]:
dfadl.head()

,DeIdentify ID,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,ScheduledTimeIn,ScheduledTimeOut,TaskCodeGroupName,HasBeenDone,Reason,Comments,NoSignature,RectangleVoiceCaptured,SignatureDate
0,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Bathroom (30),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
1,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Clean Kitchen (31),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
2,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Dust (32),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
3,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Laundry (36),Y,NaN,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM
4,Client001,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-01-03,10:00:00,13:00:00,NaN,2022-01-03,10:00:00,13:00:00,Homemaking Support:Linen Change (37),N,Performed on another day,NaN,Signature not obtained,NaN,Signature Date: 1/3/2022 1:00:02 PM


In [ ]:
dfadl['date'] = pd.to_datetime(dfadl['VisitDate'])
dfadl['date'] = dfadl['date'].dt.date
dfadl['month_year'] = dfadl['date'].map(lambda x: 100*x.year + x.month)

In [ ]:
tempdate = pd.to_datetime(dfcrf['Time'])
dfcrf['date'] = tempdate.dt.date
dfcrf['month_year'] = dfcrf['date'].map(lambda x: 100*x.year + x.month)

In [ ]:
dfcrf['client'] = dfcrf['Masked Client ID']
dfadl['client'] = dfadl['DeIdentify ID']

In [ ]:
mergedcrfadl = pd.merge(dfadl, dfcrf, on = ["client","month_year"])
mergedcrfadl

NameError: ignored

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#mergedcrfadl['Any_Hospitalizations_in_the_last_30_days']=mergedcrfadl['Any_Hospitalizations_in_the_last_30_days'].eq('Yes').mul(1)
#mergedcrfadl.Any_Hospitalizations_in_the_last_30_days=pd.Series(np.where(mergedcrfadl.Any_Hospitalizations_in_the_last_30_days.values == 'Yes', 1, 0),mergedcrfadl.index)
mergedcrfadl['Any Hospitalizations in the last 30 days'].replace(['No', 'Yes'],
                        [0, 1], inplace=True)
mergedcrfadl['Any Hospitalizations in the last 30 days'].unique()


array([0, 1])

In [ ]:
mergedcrfadl=mergedcrfadl.drop(['Date of Review', 'DeIdentify ID', 'Masked Client ID', 'Comments'], axis = 1)
mergedcrfadl

,ProviderID,PayerGroup,Caregiver ID,VisitDuration,ServiceTypeName,VisitDate,ActualTimeIn,ActualTimeOut,txtBreakTimesValue,VisitDate2,...,Detail any issues,Are the cargivers able to perform all the tasks on your care plan,Detail any issues.1,Are there any upcoming schedule changes?,Details of Schedule changes,Are you happy with the services being provided?,If No make notes here,Any Questions or Concerns?,Case Manager Notes,date_y
0,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
1,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
2,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
3,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
4,203236159.0,Veterans Home Care / Veteran Home Care / Veter...,Caregiver001,3 h,VA Home Care,2022-03-01,10:00:00,13:00:00,NaN,2022-03-01,...,NaN,NaN,NaN,No,NaN,Yes,NaN,ALL IS WELL,IN THE NEAR FUTURE WE WILL NEED A MALE CG FOR ...,2022-03-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99298,486278.0,DDD / DDD / DDD,Caregiver075,2 h,DDD Habilitation,2022-06-30,19:00:00,21:00:00,NaN,2022-06-30,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Everything is good with client and caregivers ...,2022-06-21
99299,486278.0,DDD / DDD / DDD,Caregiver075,2 h,DDD Habilitation,2022-06-30,19:00:00,21:00:00,NaN,2022-06-30,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Everything is good with client and caregivers ...,2022-06-21
99300,486278.0,DDD / DDD / DDD,Caregiver075,2 h,DDD Habilitation,2022-06-30,19:00:00,21:00:00,NaN,2022-06-30,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Everything is good with client and caregivers ...,2022-06-21
99301,486278.0,DDD / DDD / DDD,Caregiver075,2 h,DDD Habilitation,2022-06-30,19:00:00,21:00:00,NaN,2022-06-30,...,NaN,NaN,NaN,No,NaN,Yes,NaN,NONE,Everything is good with client and caregivers ...,2022-06-21


In [ ]:
mergedcrfadl=mergedcrfadl.drop(['ProviderID', 'PayerGroup', 'VisitDuration','VisitDate','txtBreakTimesValue', 'VisitDate2', 'ScheduledTimeIn','Date and Place of last HbA1C Check',
       'Date and place of last Mammogram (if female)',
       'Date and place of last Colorectal Exam',
       'Date and place of last time you had your cholesterol checked',
       'Date and place of last Bone Density Scan',
       'Does your health plan meet your needs?',
       'Has the Care Plan been reviewed with the Client',
       'Any changes to current care plan', 'Notes on changes to Care Plan',
       'Do your caregivers arrive on time?', 'Details of timekeeping issues',
       'Does your Caregiver carry out the care plan satisfactory',
       'Detail any issues',
       'Are the cargivers able to perform all the tasks on your care plan',
       'Detail any issues.1', 'Are there any upcoming schedule changes?',
       'Details of Schedule changes',
       'Are you happy with the services being provided?',
       'If No make notes here', 'Any Questions or Concerns?',
       'Case Manager Notes', 'date_y', 'Notes on changes to medical conditions','Notes on new medical devises', 'Any changes to Mental Status',
       'Notes on changes to mental status', 'How is your vision?',
       'How is your hearing', 'Do you take your medications',
       'Do you fill all your prescriptions?',
       'How would you describe your diet',
       'How much water do you drink in a day?','Note on Hospitalization (date and reason)',
       'How can we help prevent a return to hospital?','Which Review is this',
       'Insurance',
       'Have you fallen in:', 'Did your fall result in',
       'Matrix (Do you worry about falling?)',
       'Matrix (Do you have a Living Will?)', 'Matrix (Do you have a  DNR?)',
       'Do you have a POA', 'Do you need help with DNR, Living Will or POA',
       'Note on help needed?',
       'Are there any changes to your home environment since the last Intake/Review?',
       'Condition of the Home',
       'Has anyone moved in our out in the last 30 days?',
       'Are you receiving Home Health Services', 'If so from which Agency',
       'Are you expecting a Nurse or Therapist to visit you at home?',
       'What will the Nurse/Therapist do?',
       'Any changes to Transportation needs?', 'Notes on transportation needs',
       'Have you had any new DME equipment in the last 90 days?',
       'Notes on new DME equipment', 'Do you need any DME Equipment?',
       'Notes on DME Equipment needed',
       'Any new medical devises in the past 90 days', 'How do you sleep',
       'What do you do about your medical appointments','Where and when did you have your flu shot?','Caregiver ID', 'ServiceTypeName', 'ActualTimeIn', 'ActualTimeOut',
       'ScheduledTimeOut', 'Reason','NoSignature', 'RectangleVoiceCaptured', 'SignatureDate', 'date_x'], axis = 1)
mergedcrfadl.columns

Index(['client', 'month_year', 'Time', 'COVID -19',
       'Any Hospitalizations in the last 30 days',
       'Any new medical conditions in the last 90 days',
       'Have you had your flu shot this year?'],
      dtype='object')

In [ ]:
mergedcrfadl=mergedcrfadl.drop(['COVID -19','Have you had your flu shot this year?','Any new medical conditions in the last 90 days']
mergedcrfadl_data = pd.get_dummies(mergedcrfadl, columns = ['client','TaskCodeGroupName', 'HasBeenDone']

In [ ]:
arima_fit = auto_arima(mergedcrfadl_data['Any Hospitalizations in the last 30 days'], trace = True, suppress_warnings=True)
arima_fit.summary()

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-493784.525, Time=172.39 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-166420.938, Time=18.11 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-193201.959, Time=8.60 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-240657.593, Time=24.96 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-166422.938, Time=3.50 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-263098.936, Time=92.22 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-451107.534, Time=37.59 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-464561.665, Time=216.74 sec
